## ME9 closer look
At first I thought there was a beautiful transition from neural crest to mesenchyme in this sample. But the more I look at it the more I feel that it is false.  
There are two ectomesenchymal clusters here, one Alx positive and the other Dlx. They connect both to a Sox10 cluster.   
However, the markers in the Sox10 cluster lead me to believe that these cells are already predisposed to becoming Glia.  
The actual mesenchymal clusters instead coming from the previous stage. Sadly it seems that the exact moment that these cells transition to mesenchyme hasn't been captured.  
![](../markdown_images/mouse/mm10/markdown_plots/ME9_closer_umap.png)  



In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import anndata as ad
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output

import pandas as pd

import os
import sys
import time
import gc
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
##mouse mm10
start_time=time.strftime("%Y_%m_%d-%I_%M_%S_%p")
print('start time:',start_time)

sample_names = 'ME9' #specify the sample names
species = 'mouse' #specify the species
genome = 'mm10' #specify the genome
output_prefix = 'h5ad_files/' #specify the location of the cellranger output
adata = sc.read(output_prefix+species+'/'+genome+'/'+sample_names+'_after_filtering.h5ad')

In [ ]:
## code for any markdown figures ##
output_dir = 'markdown_images/'+species+'/'+genome+'/markdown_plots/'
!mkdir -p {output_dir}

i='ME9_closer'
plt.rcParams['figure.figsize'] = [4,3]
ax = sc.pl.umap(adata, color = ['Sox10','leiden_post_QC','Dlx2','Dlx1','Alx3','Tfap2b','Plp1','Insc'], cmap = reds, vmin = 0.05,ncols = 2,show = False)
plt.savefig(output_dir+i+'_umap.png', dpi = 80,bbox_inches='tight')
plt.close()

In [ ]:
adata.obs['leiden_mesen'] = list(adata.obs['leiden_post_QC'])
adata.obs.loc[adata.obs['leiden_mesen'].isin(['12','11','5','17']), 'leiden_mesen'] = "mesen"

In [ ]:
adata.obsm['X_umap'][:, 0]

In [ ]:
dict(zip(list(adata.obs['leiden_post_QC'].cat.categories), adata.uns['leiden_post_QC_colors']))

In [ ]:
category2 = 'leiden_lowres'

colordict2 = dict(zip(list(adata.obs[category2].cat.categories), adata.uns[category2+'_colors']))

plt.scatter(adata.obsm['X_umap'][:, 0],adata.obsm['X_umap'][:, 1],
            c=adata.obs[category2].map(colordict2),
            s=200,
                      )

plt.scatter(adata.obsm['X_umap'][:, 0],adata.obsm['X_umap'][:, 1],
            c='white',
            s=60,
                      )

category = 'leiden_post_QC'

colordict = dict(zip(list(adata.obs[category].cat.categories), adata.uns[category+'_colors']))

plt.scatter(adata.obsm['X_umap'][:, 0],adata.obsm['X_umap'][:, 1],
            c=adata.obs[category].map(colordict),
            s=2,
                      )
handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=8) for k, v in colordict.items()]
plt.legend(handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left',ncol=3)
plt.xticks([], [])
plt.yticks([], [])
plt.xlabel("UMAP1")
plt.ylabel("UMAP2")
plt.title(category)

In [ ]:
sc.tl.leiden(adata,resolution = .1, key_added = 'leiden_lowres')
sc.pl.umap(adata, color = 'leiden_lowres')

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_lowres', groups=['0'], reference='1', method='wilcoxon')
gene_df = sc.get.rank_genes_groups_df(adata,group = None)
gene_df = gene_df[gene_df['logfoldchanges']>2]
gene_df = gene_df[gene_df['scores']>5]
gene_df

In [ ]:
sc.pl.umap(adata, color = ['Irx2','En1','Fgf18','Wnt1','Otx2','Hoxa2','Hoxb2','Hoxb1','Hoxa3','Foxg1','Pax5','Cyp26b1','Cyp26a1','Cyp26c1','Egr2'], cmap = reds, vmin = 0.05)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_lowres', method='wilcoxon',)
gene_df = sc.get.rank_genes_groups_df(adata,group = None)
gene_df = gene_df[gene_df['logfoldchanges']>3]
gene_df = gene_df[gene_df['scores']>5]
gene_df

In [ ]:
top5 = []
for cluster in list(gene_df['group'].cat.categories):
    top5 = top5 + list(gene_df[gene_df['group']==cluster].head(5).names)

sc.pl.umap(adata, color = top5, cmap = reds, vmin = 0.05, ncols = 3)

In [ ]:
sc.pl.umap(adata, color = ['Sox10','leiden_post_QC','Tfap2b','leiden_mesen','Alx3','Dlx2','Grrp1','Pax7','Car4','Insc'],cmap = reds, vmin = 0.05, ncols = 2)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_post_QC', method='wilcoxon',)

In [ ]:
gene_df = sc.get.rank_genes_groups_df(adata,group = '24')


In [ ]:
gene_df = gene_df[gene_df['logfoldchanges']>5]
gene_df = gene_df[gene_df['scores']>5]
gene_df

In [ ]:
sc.pl.umap(adata, color = ['Pax2','Pax8'], cmap = reds, vmin = 0.05, ncols = 3)

In [ ]:
plt.rcParams['figure.figsize'] = [5,4]
sc.pl.umap(adata, color = gene_df.head(30).names, cmap = reds, vmin = 0.05, ncols = 3)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_mesen', groups=['mesen'], reference='3', method='wilcoxon')

In [ ]:
gene_df = sc.get.rank_genes_groups_df(adata,group = None)
gene_df = gene_df[gene_df['logfoldchanges']>2]
gene_df = gene_df[gene_df['scores']>8]
gene_df

In [ ]:
plt.rcParams['figure.figsize'] = [5,4]
sc.pl.umap(adata, color = gene_df.head(30).names, cmap = reds, vmin = 0.05, ncols = 3)